In [ ]:
from imports import *
from functions_uc import *

In [ ]:
train_series_dict_full,valid_series_dict_full,resampled_df,scaler = data_processing_training_scale(data_to_val,value_to_predict,resample_rate,min_ts_length,imputation,max_train_length,prediction_length,padding, scale_series)


In [ ]:
ensemble = TimeSeriesPredictor.load("./autogluon_ensemble")
chronos= TimeSeriesPredictor.load("./hronos")
local = TimeSeriesPredictor.load("./baseline")
timellm_fine_tuned = NeuralForecast.load(path='./time_llm_finetuned/')
timellm_zero_shot = NeuralForecast.load(path='./time_llm_zeroshot/')
nbeats_model = NBEATSModel.load("./nbeats.pth.tar")
lgbm_model = LightGBMModel.load("./lgbm.pth.tar")

model_name=model_name
output_dir_forecasting = "./results/"
output_dir_classification = "./results/classification/"
forecasting_filename = "model_name.csv"
classification_filename = "model_name.csv"

# Evaluate Autogluon models
predictions,actuals, train_data, test_data, merged_data,actual_values,predicted_values = autogluon_data_ci_roc(model_name,output_dir_forecasting,output_dir_classification,forecasting_filename,classification_filename,resampled_df, model, scaler,prediction_length,value_to_predict)

# Evaluate TimeLLM models
predictions,actuals, merged_data,actual_value,predicted_value = timellm_data_ci_roc_(model_name,output_dir_forecasting,output_dir_classification,forecasting_filename,classification_filename,resampled_df, model, scaler,prediction_length,value_to_predict)

# Evaluate DARTS models
predicted_values, actual_values, keys, pred,actual_values_np,predicted_values_np  = predict_proba_ci_roc(output_dir_classification,classification_filename,output_dir_forecasting,forecasting_filename,model, train_series_dict_full, valid_series_dict_full, prediction_length,scaler)

# Evaluate TimesFM models
predictions,actuals, merged_data,actual_fever,predicted_fever = timesfm_data_ci_roc_(model_name,output_dir_forecasting,output_dir_classification,forecasting_filename,classification_filename,resampled_df, tfm, scaler,prediction_length,value_to_predict)